# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


'The new Blackwell GPU features two dies, NVLink-5, and Transformer Engine 2.0 for generative AI. It has a 10 TB/s interconnect and supports HBM3e memory.'

## Regular search

In [3]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [5]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

returning previous results due to exception reaching ddg.
https://weather.com/weather/today/l/USCA0987:1:US
https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8


In [6]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [ ]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

In [8]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/today/l/USCA0987:1:US


Recent Locations Weather Forecasts Radar & Maps News & Media Products & Account Lifestyle Specialty Forecasts San Francisco, CA Weather Today in San Francisco, CA 6:30 am 7:55 pm Hourly Weather - San Francisco, CA Now Sunny 5 pm Sunny 6 pm Sunny 7 pm Sunny Don't Miss Seasonal Hub 10 Day Weather - San Francisco, CA Tonight Night Clear to partly cloudy. Low 56F. Winds WSW at 10 to 20 mph. Thu 21 Day Partly cloudy skies. High around 75F. Winds WSW at 10 to 20 mph. Night Partly cloudy skies during the evening will give way to cloudy skies overnight. Low 56F. Winds WSW at 10 to 20 mph. Fri 22 Day A few passing clouds, otherwise generally sunny. High around 70F. Winds WSW at 10 to 20 mph. Night Partly cloudy skies early will give way to considerable cloudiness and fog after midnight. Low 56F. Winds WSW at 10 to 20 mph. Sat 23 Day Partly to mostly cloudy. Expect mist and reduced visibilities at times. High 68F. Winds WSW at 10 to 20

## Agentic Search

In [9]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco De Macoris', 'region': 'Duarte', 'country': 'Dominikanische Republik', 'lat': 19.3, 'lon': -70.25, 'tz_id': 'America/Santo_Domingo', 'localtime_epoch': 1755729847, 'localtime': '2025-08-20 18:44'}, 'current': {'last_updated_epoch': 1755729000, 'last_updated': '2025-08-20 18:30', 'temp_c': 32.1, 'temp_f': 89.8, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 12.8, 'wind_kph': 20.5, 'wind_degree': 136, 'wind_dir': 'SE', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 63, 'cloud': 50, 'feelslike_c': 36.8, 'feelslike_f': 98.2, 'windchill_c': 31.4, 'windchill_f': 88.6, 'heatindex_c': 35.9, 'heatindex_f': 96.7, 'dewpoint_c': 22.5, 'dewpoint_f': 72.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.4, 'gust_mph': 16.9, 'gust_kph': 27.2}}


In [10]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco De Macoris",
        "region": "Duarte",
        "country": "Dominikanische Republik",
        "lat": 19.3,
        "lon": -70.25,
        "tz_id": "America/Santo_Domingo",
        "localtime_epoch": 1755729847,
        "localtime": "2025-08-20 18:44"
    },
    "current": {
        "last_updated_epoch": 1755729000,
        "last_updated": "2025-08-20 18:30",
        "temp_c": 32.1,
        "temp_f": 89.8,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 12.8,
        "wind_kph": 20.5,
        "wind_degree": 136,
        "wind_dir": "SE",
        "pressure_mb": 1012.0,
        "pressure_in": 29.88,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 63,
        "cloud": 50,
        "feelslike_c": 36.8,
        "feelslike_f": 98.2,
        "windchill_c": 31.4,
 

<img src="./google_sample.png" width="800" height="600">